# ***YOLOv12 + R-ELAN + BiFPN***

In [3]:
# Install required packages (for Google Colab)
!pip install -q ultralytics opencv-python roboflow pyyaml

import os
from pathlib import Path
import yaml
import torch
import torch.nn as nn
import inspect
import re
from roboflow import Roboflow
from ultralytics import YOLO
from ultralytics.nn import tasks

# Custom BiFPN Concat module
class BiFPN_Concat2(nn.Module):
    def __init__(self, dimension=1):
        super(BiFPN_Concat2, self).__init__()
        self.d = dimension
        self.w = nn.Parameter(torch.ones(2, dtype=torch.float32), requires_grad=True)
        self.epsilon = 0.0001

    def forward(self, x):
        w = self.w
        weight = w / (torch.sum(w, dim=0) + self.epsilon)
        x = [weight[0] * x[0], weight[1] * x[1]]
        return torch.cat(x, self.d)

# Register custom module
tasks.BiFPN_Concat2 = BiFPN_Concat2

# Patch parse_model to handle BiFPN_Concat2 like Concat
source = inspect.getsource(tasks.parse_model)
pattern = r'(\s*)else:\s*\n(\s*)c2\s*=\s*ch\s*\[f\](?=\s|$)'
replacement = r'\1elif m is BiFPN_Concat2:\n\2c2 = sum(ch[x] for x in f)\n\1else:\n\2c2 = ch[f]'
modified_source = re.sub(pattern, replacement, source)
exec(modified_source, tasks.__dict__)

# Konfigurasi Roboflow
API_KEY = "QOd5ldAdjiaehHn5m6WC"
WORKSPACE = "dentalogic8"
PROJECT_ID = "dental-caries-7kttb"
VERSION_NUM = 10

# Download Dataset
rf = Roboflow(api_key=API_KEY)
project = rf.workspace(WORKSPACE).project(PROJECT_ID)
dataset = project.version(VERSION_NUM).download("yolov12")

DATASET_DIR = Path(f"/content/dental-caries-{VERSION_NUM}")
DATA_YAML = DATASET_DIR / "data.yaml"

# Verifikasi download
if DATA_YAML.exists():
    print(f"✅ Dataset berhasil diunduh: {DATA_YAML}")
    print(f"Struktur dataset: {os.listdir(DATASET_DIR)}")
else:
    raise FileNotFoundError("Dataset gagal diunduh. Periksa koneksi dan kredensial Roboflow.")

# YOLO12n YAML configuration with BiFPN neck
yolo12n_yaml_content = """
# Ultralytics YOLO12n object detection model with P3/8 - P5/32 outputs (BiFPN Neck)

# Parameters
nc: 7  # number of classes
scales:  # model compound scaling constants
  n: [0.50, 0.25, 1024]  # YOLO12n
  s: [0.50, 0.50, 1024]  # YOLO12s
  m: [0.50, 1.00, 512]   # YOLO12m
  l: [1.00, 1.00, 512]   # YOLO12l
  x: [1.00, 1.50, 512]   # YOLO12x

# Backbone
backbone:
  - [-1, 1, Conv, [64, 3, 2]]    # 0-P1/2
  - [-1, 1, Conv, [128, 3, 2]]   # 1-P2/4
  - [-1, 2, C3k2, [256, False, 0.25]]
  - [-1, 1, Conv, [256, 3, 2]]   # 3-P3/8
  - [-1, 2, C3k2, [512, False, 0.25]]
  - [-1, 1, Conv, [512, 3, 2]]   # 5-P4/16
  - [-1, 4, A2C2f, [512, True, 4]]
  - [-1, 1, Conv, [1024, 3, 2]]  # 7-P5/32
  - [-1, 4, A2C2f, [1024, True, 1]]  # 8

# Head (neck modified to BiFPN)
head:
  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]  # 9
  - [[-1, 6], 1, BiFPN_Concat2, [1]]            # 10
  - [-1, 2, A2C2f, [512, False, -1]]            # 11
  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]  # 12
  - [[-1, 4], 1, BiFPN_Concat2, [1]]            # 13
  - [-1, 2, A2C2f, [256, False, -1]]            # 14
  - [-1, 1, Conv, [256, 3, 2]]                  # 15
  - [[-1, 11], 1, BiFPN_Concat2, [1]]           # 16
  - [-1, 2, A2C2f, [512, False, -1]]            # 17
  - [-1, 1, Conv, [512, 3, 2]]                  # 18
  - [[-1, 8], 1, BiFPN_Concat2, [1]]            # 19
  - [-1, 2, C3k2, [1024, True]]                 # 20
  - [[14, 17, 20], 1, Detect, [nc]]             # 21
"""

# Save YOLO12n YAML to file
yaml_path = Path("/content/yolo12n_r-elan_bifpn.yaml")
with open(yaml_path, "w") as f:
    yaml.dump(yaml.safe_load(yolo12n_yaml_content), f)
print(f"✅ YAML model saved: {yaml_path}")

# Inisialisasi model
model = YOLO(str(yaml_path))

def train_model(model, data_yaml, project_name="train_icdas_r-elan_bifpn", epochs=50):
    results = model.train(
        data=data_yaml,
        imgsz=640,
        epochs=epochs,
        batch=16,
        device=0,
        deterministic=True,
        project="runs/pure",
        name=project_name,
        pretrained=False
    )
    return results

train_results = train_model(model, DATA_YAML)
print("✅ Training selesai! Model terbaik: runs/pure/train_icdas_bifpn/weights/best.pt")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to dental-caries-9 in yolov12:: 100%|██████████| 13112/13112 [00:02<00:00, 5848.74it/s]


✅ Dataset berhasil diunduh: /content/dental-caries-9/data.yaml
Struktur dataset: ['train', 'valid', 'README.dataset.txt', 'data.yaml', 'README.roboflow.txt', 'test']
✅ YAML model saved: /content/yolo12m_bifpn.yaml
Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dental-caries-9/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mod

KeyboardInterrupt: 